Overscan Subtraction Code for CCD (with science image dimensions 2148x2048)
Adapted from the guide of Dr. Colette Salyk @ https://github.com/csalyk/phot_tutorial/blob/master/phot_tutorial.ipynb


In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc as ccdp
from ccdproc import trim_image
from ccdproc import subtract_overscan
import os
from astropy.nddata import CCDData
from skimage.registration import phase_cross_correlation
import pickle as pickle
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.time import Time
import pandas as pd

In [17]:
raw_dir='/Users/bpiotrowski/Desktop/24c04/raw'  #Your Raw Directory
overscan_dir='/Users/bpiotrowski/Desktop/24c04/overscan_sub'  #Your Overscan Directory

In [18]:
def overscan_subtraction(infiles, outdir):
    if(not outdir.endswith('/')): outdir=outdir+'/'
    if(not os.path.exists(outdir)): os.mkdir(outdir)
    
    #Scan inbox directory for .fit files
    for image in infiles:
        outfile=outdir+os.path.split(image)[1]  #Define output file
        print('Processing image '+image)
           
        #Generate a CCDData object from the image's data array
        hdul = fits.open(image)
        ccddata = CCDData(hdul[0].data, unit=u.adu)

        #subtract overscan row-by-row and crop region
        overscan_subtracted_data = subtract_overscan(ccddata, overscan=ccddata[:, 2098:2148])
        overscan_subtracted_trimmed_data = np.asarray(trim_image(overscan_subtracted_data[:, 50:2098]))    
            
        #Update header information for image width
        header = hdul[0].header
        header['NAXIS1'] = 2048
        header['IMAGEW'] = 2048
    
        #Save data to outbox directory.
        hdul[0].data = overscan_subtracted_trimmed_data
        hdul.writeto(outfile,overwrite=True)

In [19]:
#Create the list of input files
indir='/Users/bpiotrowski/Desktop/24c04/raw'  #Your Raw Directory
filelist=[]
for file in os.listdir(indir):
    if (file.endswith("z.fit") or file.endswith("d.fit") or file.endswith(".fit")):  #Updates biases, darks, and science images (adjust this one)
        filelist.append(os.path.join(indir, file))

#Define the output directory
outdir='/Users/bpiotrowski/Desktop/24c04/overscan_sub'  #Your Overscan Directory

In [20]:
overscan_subtraction(filelist, outdir)

Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat R2-005r.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat V-003v.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat B-002b.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat B-003b.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat V-002v.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat R-001r.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat R2-004r.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat H-006h.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat B-004b.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat R2-003r.fit
Processing image /Users/bpiotrowski/Desktop/24C04_flats/raw/24C04 DomeFlat R2-002r.fit
Processing image /Users/bpiotrowski/Desktop/24C04_